***Can only retrieve up to 1000 data points from Flipside API***

In [1]:
import pandas as pd
import numpy as np 
import requests
import json
import time
from flipside import Flipside
import os
import traceback
from dotenv import load_dotenv
import datetime as dt
import plotly.express as px
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
#from prophet import Prophet
from dash import Dash, html, dcc, Input, Output, State, callback
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from dash import dash_table



from sql.sql_scripts import mints_query, sales_query, eth_price_query

In [2]:
load_dotenv()

True

In [3]:
pd.options.display.float_format = '{:,.2f}'.format


In [4]:
opensea_api_key = os.getenv('opensea_api_key')

***Listing Data***

In [5]:
def fetch_listings(api_key, delay_between_requests=1):
    base_url = "https://api.opensea.io/api/v2/listings/collection/3dns-powered-domains/all"
    headers = {
        "accept": "application/json",
        "x-api-key": api_key
    }
    params = {"limit": 100} 

    listings = []
    page_count = 0

    while True:
        response = requests.get(base_url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            fetched_listings = data.get("listings", [])
            listings.extend(fetched_listings)
            page_count += 1
            
            # Extract and print the cursor
            next_cursor = data.get("next")
            print(f"Page {page_count}, Cursor: {next_cursor}, Listings Fetched: {len(fetched_listings)}")
            
            if next_cursor:
                params['next'] = next_cursor  # Update the 'next' parameter for the next request
            else:
                break  # No more pages to fetch
                
            # Implementing delay
            time.sleep(delay_between_requests)
            
        else:
            print(f"Failed to fetch data: {response.status_code}")
            break

    print(f"Total pages fetched: {page_count}")
    print(f"Total listings fetched: {len(listings)}")
    
    df = pd.DataFrame(listings)
    return df

***Descriptions***

In [6]:
def save_last_identifier(identifier):
    with open("last_identifier.txt", "w") as file:
        file.write(identifier)

def load_last_identifier():
    try:
        with open("last_identifier.txt", "r") as file:
            return file.read().strip()
    except FileNotFoundError:
        return None

In [7]:
def fetch_all_descriptions(api_key, delay_between_requests=1):
    base_url = "https://api.opensea.io/api/v2/collection/3dns-powered-domains/nfts"
    headers = {
        "accept": "application/json",
        "x-api-key": api_key
    }
    params = {"limit": 200}

    all_descriptions = []

    page_count = 0
    last_identifier = load_last_identifier()

    while True:
        if last_identifier:
            params['last_identifier'] = last_identifier

        response = requests.get(base_url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            fetched_descriptions = data.get("nfts", [])
            
            if not fetched_descriptions:
                break

            # Process only name and identifier for each description
            for description in fetched_descriptions:
                processed_description = {
                    "name": description.get('name'),
                    "identifier": description.get('identifier')
                }
                all_descriptions.append(processed_description)
            
            # Update the last_identifier to the latest one fetched
            last_identifier = fetched_descriptions[-1].get('identifier')
            save_last_identifier(last_identifier)
            
            page_count += 1
            next_cursor = data.get("next")
            print(f"Page {page_count}, Cursor: {next_cursor} Descriptions Fetched: {len(fetched_descriptions)}, total fetched: {len(all_descriptions)}")
            
            if next_cursor:
                params['next'] = next_cursor
            else:
                break  # No more pages to fetch

            time.sleep(delay_between_requests)
        else:
            print(f"Failed to fetch data: {response.status_code}")
            break

    print(f"Total pages fetched: {page_count}, Total descriptions fetched: {len(all_descriptions)}")
    
    # Save the processed descriptions to a file
    df = pd.DataFrame(all_descriptions)
    return df

***Events***

In [8]:
import json
import os

def save_last_timestamp(event_type, timestamp):
    data = {}
    if os.path.exists("last_timestamps.json"):
        with open("last_timestamps.json", "r") as file:
            data = json.load(file)
    data[event_type] = timestamp
    with open("last_timestamps.json", "w") as file:
        json.dump(data, file)

def load_last_timestamp(event_type):
    if os.path.exists("last_timestamps.json"):
        with open("last_timestamps.json", "r") as file:
            data = json.load(file)
        return data.get(event_type, None)
    return None

In [9]:
def fetch_event_type(api_key, event_type, all_events, params, headers):
    base_url = f"https://api.opensea.io/api/v2/events/collection/3dns-powered-domains"
    params['event_type'] = event_type
    
    # Load the last timestamp/identifier
    last_timestamp = load_last_timestamp(event_type)
    if last_timestamp:
        params['occurred_after'] = last_timestamp
    
    page_count = 0
    while True:
        response = requests.get(base_url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            fetched_events = data.get("asset_events", [])
            all_events.extend(fetched_events)
            
            if fetched_events:
                # Update the last timestamp/identifier to the latest one fetched
                last_event_time = fetched_events[-1].get("created_date")
                save_last_timestamp(event_type, last_event_time)
            
            page_count += 1
            next_cursor = data.get("next")
            print(f"Fetching {event_type}: Page {page_count}, Events Fetched: {len(fetched_events)}, Total Events: {len(all_events)}, next cursor: {next_cursor}")
            
            if next_cursor:
                params['next'] = next_cursor
            else:
                break  # No more pages to fetch

            time.sleep(1)  # Delay between requests
        else:
            print(f"Failed to fetch {event_type} data: HTTP {response.status_code}, Response: {response.text}")
            break

def fetch_all_events(api_key):
    headers = {
        "accept": "application/json",
        "x-api-key": api_key
    }
    params = {
        "limit": 50  # Adjust the limit as needed
    }

    all_events = []

    # Fetch listings
    fetch_event_type(api_key, "listing", all_events, params.copy(), headers)

    # Fetch sales
    fetch_event_type(api_key, "sale", all_events, params.copy(), headers)

    # Save the fetched events to a DataFrame
    print(f"Total events fetched: {len(all_events)}")
    df = pd.DataFrame(all_events)
    return df 

***Flipside Data***

In [10]:
flipside_api_key = os.getenv("FLIPSIDE_API_KEY")
flipside = Flipside(flipside_api_key, "https://api-v2.flipsidecrypto.xyz")

In [11]:
def flipside_api_results(query):
  query_result_set = flipside.query(query)
  # what page are we starting on?
  current_page_number = 1

  # How many records do we want to return in the page?
  page_size = 1000

  # set total pages to 1 higher than the `current_page_number` until
  # we receive the total pages from `get_query_results` given the 
  # provided `page_size` (total_pages is dynamically determined by the API 
  # based on the `page_size` you provide)

  total_pages = 2


  # we'll store all the page results in `all_rows`
  all_rows = []

  while current_page_number <= total_pages:
    results = flipside.get_query_results(
      query_result_set.query_id,
      page_number=current_page_number,
      page_size=page_size
    )

    total_pages = results.page.totalPages
    if results.records:
        all_rows = all_rows + results.records
    
    current_page_number += 1

  return pd.DataFrame(all_rows)

***Data Retrieval/Processing***

mint_df = flipside_api_results(mints_query)
mint_df.to_csv('data/mint_data.csv', index=False)


In [12]:
mint_df = pd.read_csv('data/mint_data.csv')
mint_df.drop(columns=['__row_index'], inplace=True)
mint_df.columns

Index(['day', 'tokenid', 'tx_hash'], dtype='object')

sales_df = flipside_api_results(sales_query)
sales_df.to_csv('data/sales_data.csv', index=False)


In [13]:
sales_df = pd.read_csv('data/sales_data.csv')
sales_df.drop(columns=['__row_index'], inplace=True)
sales_df.columns

Index(['day', 'tokenid', 'tx_hash', 'price'], dtype='object')

eth_usd_df = flipside_api_results(eth_price_query)
eth_usd_df.to_csv('data/eth_usd.csv', index=False)


In [14]:
eth_usd_df = pd.read_csv('data/eth_usd.csv')
eth_usd_df.drop(columns=['__row_index'], inplace=True)
eth_usd_df.columns

Index(['day', 'price'], dtype='object')

events_df = fetch_all_events(api_key= opensea_api_key)
events_df.to_json('data/events_data.json', orient='records', date_format='iso')



In [15]:
events_df = pd.read_json('data/events_data.json', orient='records')
events_df = events_df.dropna()

descriptions_df = fetch_all_descriptions(api_key= opensea_api_key)
descriptions_df.to_json('data/descriptions_data.json', orient='records', date_format='iso')

In [16]:
descriptions_df = pd.read_json('data/descriptions_data.json', orient='records')
descriptions_df = descriptions_df.dropna()

descriptions_df['name'].tail(20)

20476             rxare.com
20477          carrabre.com
20478          petbitch.com
20479     storagecorner.org
20480       nftsarereal.com
20481              aox.link
20482           kenzlee.com
20483          anaclare.org
20484          debanked.net
20485         shoemaker.wtf
20486      wassiehunter.com
20487     tokentroopers.xyz
20488            mudbox.xyz
20489     beallclothing.com
20490          brandley.xyz
20491        tameimpala.xyz
20492          efferium.org
20493           wizkika.com
20494    onchaindomains.org
20495              3dns.box
Name: name, dtype: object

listings_df = fetch_listings(api_key= opensea_api_key, delay_between_requests=1)
listings_df.to_json('data/listings_data.json', orient='records', date_format='iso')

In [17]:
listings_df = pd.read_json('data/listings_data.json', orient='records')
listings_df = listings_df.dropna()

In [18]:
def unpack_protocol_data(row):
    protocol_data = row['protocol_data']
    parameters = protocol_data.get('parameters', {})
    consideration = parameters.get('consideration', [{}])
    offer = parameters.get('offer', [{}])
    price = row['price']['current']
    
    chain = row['chain']
    order_hash = row['order_hash']
    currency = price.get('currency')
    price_string = price.get('value')
    price_in_eth = float(price_string) / (10 ** price.get('decimals', 18))
    primary_recipient = consideration[0].get('recipient') if consideration else None
    identifier_or_criteria = offer[0].get('identifierOrCriteria') if offer else None
    start_time = parameters.get('startTime')
    end_time = parameters.get('endTime')
    
    return pd.Series([
        chain, order_hash, currency, price_string, price_in_eth, 
        primary_recipient, identifier_or_criteria, start_time, end_time
    ])

In [19]:
unpacked_columns = listings_df.apply(unpack_protocol_data, axis=1)
unpacked_columns.columns = [
    'chain', 'order_hash', 'currency', 'price_string', 'price_in_eth', 
    'primary_recipient', 'identifier_or_criteria', 'start_time', 'end_time'
]
listings_df = pd.concat([listings_df, unpacked_columns], axis=1)
listings_df = listings_df.drop(columns=['protocol_data'])


In [20]:
listings_df['identifier_or_criteria'] = listings_df['identifier_or_criteria'].astype(float)
listings_df.rename(columns={'identifier_or_criteria':'tokenid'}, inplace=True)

In [21]:
listings_df.drop(columns=['price_string','currency','primary_recipient','chain','price','protocol_address','type'], inplace=True)

In [22]:
descriptions_df.rename(columns={'identifier':'tokenid'}, inplace=True)

In [23]:
descriptions_df['tokenid'] = descriptions_df['tokenid'].astype(float)

In [24]:
listings_with_names = listings_df.merge(descriptions_df, how='left', on='tokenid')
listings_with_names

,order_hash,order_hash,price_in_eth,tokenid,start_time,end_time,name
0,0x09079c53f0de1b0f74eebb122d7dde6f2b15597c2493...,0x09079c53f0de1b0f74eebb122d7dde6f2b15597c2493...,4.52,"35,680,721,375,902,822,274,595,445,380,444,558,...",1706123455,1721844648,postoffice.box
1,0x62f70b03d872b0ad28daf741f24172dd3e0a07ae830d...,0x62f70b03d872b0ad28daf741f24172dd3e0a07ae830d...,1.50,"14,048,860,107,254,168,745,279,621,511,972,539,...",1706123508,1721844704,outofthe.box
2,0x49e904f3f1968a867549eb12af481ed707206deaaab5...,0x49e904f3f1968a867549eb12af481ed707206deaaab5...,15.00,"83,025,402,216,097,860,850,439,774,554,040,107,...",1706123545,1721844742,gym.box
3,0xcb2de7a4cd7972fb0799f115f5520d07eda3e9020256...,0xcb2de7a4cd7972fb0799f115f5520d07eda3e9020256...,32.00,"102,970,000,717,591,218,890,627,833,037,729,437...",1706149250,1721870447,322.box
4,0x8ec799a93294ec3d13926e54aa712112d8f14f504d47...,0x8ec799a93294ec3d13926e54aa712112d8f14f504d47...,6.90,"39,332,446,229,392,020,518,394,952,709,307,862,...",1706152363,1721877156,shipping.box
...,...,...,...,...,...,...,...
713,0x8902337cb70172fc5465d11f7cb4c78c7af9c2b35dcd...,0x8902337cb70172fc5465d11f7cb4c78c7af9c2b35dcd...,0.00,"2,791,407,270,785,156,193,583,568,783,839,770,1...",1721842873,1737256913,memecoins-on.chain.box
714,0xf7da2a7dd645b3e9afb8153608258b941ce39f6330c9...,0xf7da2a7dd645b3e9afb8153608258b941ce39f6330c9...,0.00,"8,168,332,643,742,886,007,163,286,795,415,474,8...",1721842888,1737269207,memes.super.box
715,0x49fa52810cc6d37e010ad5edda77ad110ba7748cf3f9...,0x49fa52810cc6d37e010ad5edda77ad110ba7748cf3f9...,0.00,"8,168,332,643,742,886,007,163,286,795,415,474,8...",1721842983,1737394962,memes.super.box
716,0x27d3dfc8eba3f66568466a533d39cfd9e1886c41d5de...,0x27d3dfc8eba3f66568466a533d39cfd9e1886c41d5de...,0.00,"2,791,407,270,785,156,193,583,568,783,839,770,1...",1721843042,1737395027,memecoins-on.chain.box


In [25]:
listings_with_names = listings_with_names.dropna()

In [26]:
box_listings = listings_with_names[listings_with_names['name'].str.endswith('.box')]
box_listings

,order_hash,order_hash,price_in_eth,tokenid,start_time,end_time,name
0,0x09079c53f0de1b0f74eebb122d7dde6f2b15597c2493...,0x09079c53f0de1b0f74eebb122d7dde6f2b15597c2493...,4.52,"35,680,721,375,902,822,274,595,445,380,444,558,...",1706123455,1721844648,postoffice.box
1,0x62f70b03d872b0ad28daf741f24172dd3e0a07ae830d...,0x62f70b03d872b0ad28daf741f24172dd3e0a07ae830d...,1.50,"14,048,860,107,254,168,745,279,621,511,972,539,...",1706123508,1721844704,outofthe.box
2,0x49e904f3f1968a867549eb12af481ed707206deaaab5...,0x49e904f3f1968a867549eb12af481ed707206deaaab5...,15.00,"83,025,402,216,097,860,850,439,774,554,040,107,...",1706123545,1721844742,gym.box
3,0xcb2de7a4cd7972fb0799f115f5520d07eda3e9020256...,0xcb2de7a4cd7972fb0799f115f5520d07eda3e9020256...,32.00,"102,970,000,717,591,218,890,627,833,037,729,437...",1706149250,1721870447,322.box
4,0x8ec799a93294ec3d13926e54aa712112d8f14f504d47...,0x8ec799a93294ec3d13926e54aa712112d8f14f504d47...,6.90,"39,332,446,229,392,020,518,394,952,709,307,862,...",1706152363,1721877156,shipping.box
...,...,...,...,...,...,...,...
713,0x8902337cb70172fc5465d11f7cb4c78c7af9c2b35dcd...,0x8902337cb70172fc5465d11f7cb4c78c7af9c2b35dcd...,0.00,"2,791,407,270,785,156,193,583,568,783,839,770,1...",1721842873,1737256913,memecoins-on.chain.box
714,0xf7da2a7dd645b3e9afb8153608258b941ce39f6330c9...,0xf7da2a7dd645b3e9afb8153608258b941ce39f6330c9...,0.00,"8,168,332,643,742,886,007,163,286,795,415,474,8...",1721842888,1737269207,memes.super.box
715,0x49fa52810cc6d37e010ad5edda77ad110ba7748cf3f9...,0x49fa52810cc6d37e010ad5edda77ad110ba7748cf3f9...,0.00,"8,168,332,643,742,886,007,163,286,795,415,474,8...",1721842983,1737394962,memes.super.box
716,0x27d3dfc8eba3f66568466a533d39cfd9e1886c41d5de...,0x27d3dfc8eba3f66568466a533d39cfd9e1886c41d5de...,0.00,"2,791,407,270,785,156,193,583,568,783,839,770,1...",1721843042,1737395027,memecoins-on.chain.box


In [27]:
events_df_copy = events_df.copy()
events_df_copy

,event_type,order_hash,order_type,chain,protocol_address,start_date,expiration_date,asset,quantity,maker,taker,payment,criteria,event_timestamp,is_private_listing,closing_date,nft,seller,buyer,transaction


In [28]:
bids = events_df_copy[events_df_copy['event_type'] == 'order']
sales = events_df_copy[events_df_copy['event_type'] == 'sale']

bids

,event_type,order_hash,order_type,chain,protocol_address,start_date,expiration_date,asset,quantity,maker,taker,payment,criteria,event_timestamp,is_private_listing,closing_date,nft,seller,buyer,transaction


In [29]:
bids['identifier'] = bids['asset'].apply(lambda x: x.get('identifier') if isinstance(x, dict) else None)
sales['identifier'] = sales['nft'].apply(lambda x: x.get('identifier') if isinstance(x, dict) else None)

In [30]:
bids['identifier'] = bids['identifier'].astype(float)
bids.rename(columns={'identifier':'tokenid'}, inplace=True) 


In [31]:
bids = bids.merge(descriptions_df, how='left', on='tokenid')

In [32]:
bids.columns

Index(['event_type', 'order_hash', 'order_type', 'chain', 'protocol_address',
       'start_date', 'expiration_date', 'asset', 'quantity', 'maker', 'taker',
       'payment', 'criteria', 'event_timestamp', 'is_private_listing',
       'closing_date', 'nft', 'seller', 'buyer', 'transaction', 'tokenid',
       'name'],
      dtype='object')

In [33]:
bids.drop(columns=['protocol_address','chain','maker','criteria','is_private_listing','closing_date','nft','seller','buyer'], inplace=True)

In [34]:
sales['identifier'] = sales['identifier'].astype(float)
sales.rename(columns={'identifier':'tokenid'}, inplace=True)


In [35]:
sales = sales.merge(descriptions_df, how='left', on='tokenid')
sales.drop(columns=['order_type', 'chain','start_date','expiration_date','asset','maker','is_private_listing','protocol_address','criteria'], inplace=True)

In [36]:
sales = sales.dropna()
bids = bids.dropna()

In [37]:
box_sales_os = sales[sales['name'].str.endswith('.box')]
box_bids_os = bids[bids['name'].str.endswith('.box')] 

In [38]:
box_bids_os.columns = [f'bid_{col}' if col != 'name' else col for col in box_bids_os.columns]

In [39]:
box_sales_os.columns = [f'sale_{col}' if col != 'name' else col for col in box_sales_os.columns]

In [40]:
box_listings_and_sales = pd.merge(box_bids_os, box_sales_os, how='inner', on='name')
box_listings_and_sales['bid_event_timestamp'] = pd.to_datetime(box_listings_and_sales['bid_event_timestamp'], unit='s')
box_listings_and_sales['sale_event_timestamp'] = pd.to_datetime(box_listings_and_sales['sale_event_timestamp'], unit='s')


In [41]:
filtered_box_listings_and_sales = box_listings_and_sales[box_listings_and_sales['sale_event_timestamp'] > box_listings_and_sales['bid_event_timestamp']]
filtered_box_listings_and_sales['time_diff'] = filtered_box_listings_and_sales['sale_event_timestamp'] - filtered_box_listings_and_sales['bid_event_timestamp']


In [42]:
avg_time_to_sell = filtered_box_listings_and_sales['time_diff'].mean()
print(avg_time_to_sell)

NaT


In [43]:
closest_listings = filtered_box_listings_and_sales.loc[filtered_box_listings_and_sales.groupby(['name', 'sale_event_timestamp'])['time_diff'].idxmin()]


In [44]:
closest_listings['sale_quantity'] = closest_listings['sale_payment'].apply(lambda x: int(x['quantity']) / 10**18)
closest_listings['listing_quantity'] = closest_listings['bid_payment'].apply(lambda x: int(x['quantity']) / 10**18)


In [45]:
closest_listings[['bid_event_timestamp','listing_quantity']].head()

,bid_event_timestamp,listing_quantity


In [46]:
closest_listings['bid_event_timestamp'] = closest_listings['bid_event_timestamp'].dt.strftime('%Y-%m-%d %H:00:00')


In [47]:
closest_listings['sale_event_timestamp'] = closest_listings['sale_event_timestamp'].dt.strftime('%Y-%m-%d %H:00:00')

In [48]:
mint_df['tokenid'] = mint_df['tokenid'].astype(float)

In [49]:
mints_with_names = pd.merge(mint_df, descriptions_df, how='left', on='tokenid')

In [50]:
mints_with_names_null = mints_with_names[mints_with_names.isnull().any(axis=1)]
print(list(mints_with_names_null['tx_hash']))

['...']


In [51]:
mints_with_names.drop_duplicates('tokenid', inplace=True)

In [52]:
mints_with_names.drop(columns=['tx_hash','tokenid'], inplace=True)

In [53]:
mints_with_names.set_index('day', inplace=True)

In [54]:
mints_with_names.index = pd.to_datetime(mints_with_names.index)
mints_with_names.dropna(inplace=True)

In [55]:
box_domains_mints = mints_with_names[mints_with_names['name'].str.endswith('.box')]


In [56]:
daily_box_mints = box_domains_mints.resample('D').count()

In [57]:
daily_box_mints.rename(columns={'name':'mints'}, inplace=True)
daily_box_mints_fig = px.bar(daily_box_mints, x=daily_box_mints.index, y='mints', title='Daily Mints')
# daily_box_mints_fig.show()

In [58]:
total_box_mints = box_domains_mints.count().iloc[0]
total_box_mints

np.int64(3927)

In [59]:
sales_with_names = pd.merge(sales_df, descriptions_df, how='left', on='tokenid')

In [60]:
sales_with_names.drop_duplicates('tokenid', inplace=True)
sales_with_names.drop(columns=['tx_hash','tokenid'], inplace=True)
sales_with_names.set_index('day', inplace=True)
sales_with_names.index = pd.to_datetime(sales_with_names.index)

In [61]:
sales_with_names

,price,name
day,,
2024-06-13 19:00:00+00:00,0.00,geusa.xyz
2024-06-13 19:00:00+00:00,0.00,NaN
2024-07-01 13:00:00+00:00,0.00,NaN
2024-07-19 05:00:00+00:00,0.00,eyeofthepanda.xyz
2024-07-19 10:00:00+00:00,0.00,bitcoindata.xyz
2024-07-22 20:00:00+00:00,0.00,understandmoney.one
2024-07-23 23:00:00+00:00,0.00,lauc.chain.box
2024-07-24 06:00:00+00:00,0.00,cryptobet.chain.box
2023-12-26 21:00:00+00:00,0.02,topdomains.finance


In [62]:
sales_with_names.dropna(inplace=True)

In [63]:
box_domains_sales = sales_with_names[sales_with_names['name'].str.endswith('.box')]
box_domains_sales

,price,name
day,,
2024-07-23 23:00:00+00:00,0.00,lauc.chain.box
2024-07-24 06:00:00+00:00,0.00,cryptobet.chain.box
2024-02-03 22:00:00+00:00,0.01,696.box
2024-02-10 19:00:00+00:00,0.04,to.box
2024-02-20 12:00:00+00:00,0.07,uae.box
2024-02-25 02:00:00+00:00,0.04,010.box
2024-02-25 03:00:00+00:00,0.04,ap.box
2024-02-25 03:00:00+00:00,0.07,40.box
2024-02-25 03:00:00+00:00,0.04,015.box


In [64]:
eth_usd_df.set_index('day', inplace=True)
eth_usd_df.index = pd.to_datetime(eth_usd_df.index)
# eth_usd_df.drop(columns=['__row_index'], inplace=True)

In [65]:
eth_usd_df.rename(columns={'price':'eth_usd'}, inplace=True)

In [66]:
box_listings['start_time'] = pd.to_datetime(box_listings['start_time'], unit='s').dt.strftime('%Y-%m-%d %H:00:00')
box_listings['end_time'] = pd.to_datetime(box_listings['end_time'], unit='s').dt.strftime('%Y-%m-%d %H:00:00')

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_9536\1480011625.py:1: FutureWarning:

The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_9536\1480011625.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_9536\1480011625.py:2: FutureWarning:

The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 

In [67]:
eth_usd_df_copy = eth_usd_df.reset_index().copy()
eth_usd_df_copy.rename(columns={'day':'start_time'}, inplace=True)

In [68]:
eth_usd_df_copy['start_time'] = pd.to_datetime(eth_usd_df_copy['start_time']).dt.tz_localize(None)

In [69]:
box_listings['start_time'] = pd.to_datetime(box_listings['start_time']) 

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_9536\1939952577.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [70]:
eth_usd_df_copy_2 = eth_usd_df_copy.copy()
eth_usd_df_copy_2.rename(columns={'start_time':'bid_event_timestamp', 'eth_usd':'eth_usd_bid'}, inplace=True)

In [71]:
eth_usd_df_copy_3 = eth_usd_df_copy.copy()
eth_usd_df_copy_3.rename(columns={'start_time':'sale_event_timestamp', 'eth_usd':'eth_usd_sale'}, inplace=True)

In [72]:
closest_listings['bid_event_timestamp'] = pd.to_datetime(closest_listings['bid_event_timestamp'])
closest_listings['sale_event_timestamp'] = pd.to_datetime(closest_listings['sale_event_timestamp'])

In [73]:
closest_listings = closest_listings.merge(eth_usd_df_copy_2, how='left', on='bid_event_timestamp')

In [74]:
closest_listings = closest_listings.merge(eth_usd_df_copy_3, how='left', on='sale_event_timestamp')

In [75]:
closest_listings['sale_usd'] = closest_listings['sale_quantity'] * closest_listings['eth_usd_sale']
closest_listings['list_usd'] = closest_listings['listing_quantity'] * closest_listings['eth_usd_bid']

In [76]:
closest_listings['percent_change'] = (closest_listings['sale_usd'] - closest_listings['list_usd']) / closest_listings['list_usd'] * 100

In [77]:
listing_price_to_sale_avg_pct_change = closest_listings['percent_change'].mean()
print(listing_price_to_sale_avg_pct_change)

nan


In [78]:

box_listings = box_listings.merge(eth_usd_df_copy, how='left', on='start_time') 

In [79]:
box_listings.drop(columns=['order_hash'], inplace=True)

In [80]:
box_listings['price_in_usd_start_time'] = box_listings['price_in_eth'] * box_listings['eth_usd']

In [81]:
box_listings.set_index('start_time', inplace=True)
box_listings_max_daily = box_listings['price_in_usd_start_time'].resample('D').max()

In [82]:
box_listings_num_daily = box_listings['name'].resample('D').count()
total_box_listings = box_listings_num_daily.sum()
total_box_listings

np.int64(501)

In [83]:
box_listings_min_daily = box_listings['price_in_usd_start_time'].resample('D').min()
box_listings_avg_daily = box_listings['price_in_usd_start_time'].resample('D').mean()

In [84]:
box_listings_max_daily.fillna(0, inplace=True)
box_listings_min_daily.fillna(0, inplace=True)
box_listings_avg_daily.fillna(0, inplace=True)

In [85]:
box_listing_data = pd.merge(box_listings_num_daily.to_frame('listings'), box_listings_max_daily.to_frame('max_price'), left_index=True,
                            right_index=True, how='inner')

In [86]:
box_listing_data = box_listing_data.merge(box_listings_min_daily.to_frame('min_price'), left_index=True,
                                          right_index=True, how='inner')

box_listing_data = box_listing_data.merge(box_listings_avg_daily.to_frame('avg_price'), left_index=True,
                                          right_index=True, how='inner')

In [87]:
monthly_listings = box_listings['name'].resample('M').count()
monthly_listings

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_9536\1847853540.py:1: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



start_time
2024-01-31     25
2024-02-29     17
2024-03-31     11
2024-04-30     88
2024-05-31    141
2024-06-30     76
2024-07-31    143
Freq: ME, Name: name, dtype: int64

In [88]:
def monthly_listings_growth_rate(listings):
    previous_month = listings.shift(1)
    growth_rate = ((listings - previous_month) / previous_month) * 100
    return growth_rate

In [89]:
listings_growth_rate = monthly_listings_growth_rate(monthly_listings)


In [90]:
listings_growth_rate.dropna(inplace=True)
listings_growth_rate

start_time
2024-02-29   -32.00
2024-03-31   -35.29
2024-04-30   700.00
2024-05-31    60.23
2024-06-30   -46.10
2024-07-31    88.16
Freq: ME, Name: name, dtype: float64

In [91]:
box_domains_sales = box_domains_sales.merge(eth_usd_df, left_index=True, right_index=True, how='left')

In [92]:
box_domains_sales['price_usd'] = box_domains_sales['price'] * box_domains_sales['eth_usd']
box_domains_sales.rename(columns={'price':'price_eth'}, inplace=True)

In [93]:
box_domains_sales.drop(columns=['eth_usd'], inplace=True)


In [94]:
box_domains_sales.sort_index(inplace=True)
box_domains_mints.sort_index(inplace=True)

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_9536\3245620862.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [95]:
box_domains_sales = box_domains_sales[['name', 'price_usd','price_eth']]


In [96]:
max_eth_sale = box_domains_sales['price_eth'].max()
max_usd_sale = box_domains_sales['price_usd'].max()

# Retrieve the corresponding timestamps
max_eth_sale_row = box_domains_sales.loc[box_domains_sales['price_eth'].idxmax()]
max_usd_sale_row = box_domains_sales.loc[box_domains_sales['price_usd'].idxmax()]

# Display the results
print(f"Maximum sale: \n {max_eth_sale_row}")


Maximum sale: 
 name           vm.box
price_usd   31,481.47
price_eth       10.00
Name: 2024-04-27 16:00:00+00:00, dtype: object


In [97]:
total_box_sales = box_domains_sales['name'].count()
print(f'total .box domain sales as of {dt.datetime.today()} : {total_box_sales}')

total .box domain sales as of 2024-07-24 13:12:01.183311 : 16


In [98]:
daily_box_sales = box_domains_sales['name'].resample('D').count()
daily_box_sales

day
2024-02-03 00:00:00+00:00    1
2024-02-04 00:00:00+00:00    0
2024-02-05 00:00:00+00:00    0
2024-02-06 00:00:00+00:00    0
2024-02-07 00:00:00+00:00    0
                            ..
2024-07-20 00:00:00+00:00    0
2024-07-21 00:00:00+00:00    0
2024-07-22 00:00:00+00:00    0
2024-07-23 00:00:00+00:00    1
2024-07-24 00:00:00+00:00    1
Freq: D, Name: name, Length: 173, dtype: int64

In [99]:
daily_box_vol = box_domains_sales['price_usd'].resample('D').sum()
cumulative_box_vol = daily_box_vol.cumsum()
cumulative_box_vol

day
2024-02-03 00:00:00+00:00       33.09
2024-02-04 00:00:00+00:00       33.09
2024-02-05 00:00:00+00:00       33.09
2024-02-06 00:00:00+00:00       33.09
2024-02-07 00:00:00+00:00       33.09
                               ...   
2024-07-20 00:00:00+00:00   33,482.18
2024-07-21 00:00:00+00:00   33,482.18
2024-07-22 00:00:00+00:00   33,482.18
2024-07-23 00:00:00+00:00   33,487.17
2024-07-24 00:00:00+00:00   33,490.27
Freq: D, Name: price_usd, Length: 173, dtype: float64

In [100]:
daily_box_sales_fig = px.bar(daily_box_sales.to_frame('sales'), x=daily_box_sales.index, y='sales', title='Daily Sales')
# daily_box_sales_fig.show()

In [101]:
latest_box_domains_sales = box_domains_sales.iloc[-10:] 
latest_box_domains_sales

,name,price_usd,price_eth
day,,,
2024-02-25 03:00:00+00:00,015.box,104.68,0.04
2024-02-25 03:00:00+00:00,75.box,269.17,0.09
2024-04-17 18:00:00+00:00,mewtwo.box,29.61,0.01
2024-04-24 04:00:00+00:00,0000.box,93.92,0.03
2024-04-27 16:00:00+00:00,vm.box,"31,481.47",10.00
2024-04-28 01:00:00+00:00,man.box,224.60,0.07
2024-04-28 07:00:00+00:00,good.box,314.61,0.10
2024-05-07 22:00:00+00:00,onyx.box,167.83,0.06
2024-07-23 23:00:00+00:00,lauc.chain.box,5.00,0.00


In [102]:
latest_box_domains_mints = box_domains_mints.iloc[-10:]
# latest_box_domains_mints

In [103]:
cumulative_box_mints = daily_box_mints.cumsum()
cumulative_box_mints.rename(columns={'mints':'cumulative mints'}, inplace=True)

In [104]:
daily_mint_metrics = daily_box_mints.merge(cumulative_box_mints, left_index=True, right_index=True, how='inner')
# daily_mint_metrics

In [105]:
daily_mint_metrics_fig = make_subplots(specs=[[{"secondary_y": True}]])

daily_mint_metrics_fig.add_trace(
    go.Bar(
        x=daily_mint_metrics.index,
        y=daily_mint_metrics['mints'],
        name='Mints'
    ),
    secondary_y=False
)

daily_mint_metrics_fig.add_trace(
    go.Scatter(
        x=daily_mint_metrics.index,
        y=daily_mint_metrics['cumulative mints'],
        name='Cumulative Mints',
        mode='lines'
    ),
    secondary_y=True
)

daily_mint_metrics_fig.update_xaxes(title_text="Date")

# daily_mint_metrics_fig.show()


In [106]:
listings_growth_rate

start_time
2024-02-29   -32.00
2024-03-31   -35.29
2024-04-30   700.00
2024-05-31    60.23
2024-06-30   -46.10
2024-07-31    88.16
Freq: ME, Name: name, dtype: float64

In [107]:
listings_growth_rate_fig = px.bar(listings_growth_rate.to_frame('Monthly Listings Growth Rate'), x=listings_growth_rate.index,
                                   y='Monthly Listings Growth Rate', title='Monthly Listings Growth Rate')

# listings_growth_rate_fig.show()

In [108]:
cumulative_box_sales = daily_box_sales.cumsum()


In [109]:
monthly_max_sold = box_domains_sales['price_usd'].resample('M').max()
monthly_min_sold = box_domains_sales['price_usd'].resample('M').min()
monthly_avg_sold = box_domains_sales['price_usd'].resample('M').mean()
monthly_volume_usd = box_domains_sales['price_usd'].resample('M').sum()
monthly_num_sold = box_domains_sales['name'].resample('M').count()

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_9536\3455025639.py:1: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_9536\3455025639.py:2: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_9536\3455025639.py:3: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_9536\3455025639.py:4: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_9536\3455025639.py:5: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [110]:
monthly_max_sold.fillna(0, inplace=True)
monthly_min_sold.fillna(0, inplace=True)
monthly_avg_sold.fillna(0, inplace=True)
monthly_volume_usd.fillna(0, inplace=True)
monthly_num_sold.fillna(0, inplace=True)

In [111]:
monthly_box_sales_metrics = pd.merge(monthly_max_sold.to_frame('max_price'), monthly_min_sold.to_frame('min_price'), left_index=True, right_index=True, how='inner')

In [112]:
monthly_box_sales_metrics = monthly_box_sales_metrics.merge(monthly_avg_sold.to_frame('avg_price'), left_index=True, right_index=True, how='inner')

In [113]:
monthly_box_sales_metrics = monthly_box_sales_metrics.merge(monthly_volume_usd.to_frame('volume_usd'), left_index=True, right_index=True, how='inner')

In [114]:
monthly_num_sold

day
2024-02-29 00:00:00+00:00    8
2024-03-31 00:00:00+00:00    0
2024-04-30 00:00:00+00:00    5
2024-05-31 00:00:00+00:00    1
2024-06-30 00:00:00+00:00    0
2024-07-31 00:00:00+00:00    2
Freq: ME, Name: name, dtype: int64

In [115]:
monthly_box_sales_metrics = monthly_box_sales_metrics.merge(monthly_num_sold.to_frame('domains sold'), left_index=True, right_index=True, how='inner')

In [116]:
monthly_box_sales_metrics

,max_price,min_price,avg_price,volume_usd,domains sold
day,,,,,
2024-02-29 00:00:00+00:00,269.17,33.09,146.27,"1,170.13",8
2024-03-31 00:00:00+00:00,0.00,0.00,0.00,0.00,0
2024-04-30 00:00:00+00:00,"31,481.47",29.61,"6,428.84","32,144.22",5
2024-05-31 00:00:00+00:00,167.83,167.83,167.83,167.83,1
2024-06-30 00:00:00+00:00,0.00,0.00,0.00,0.00,0
2024-07-31 00:00:00+00:00,5.00,3.09,4.05,8.09,2


In [117]:
daily_sales_metrics = pd.merge(cumulative_box_sales.to_frame('cumulative_sales'), daily_box_sales.to_frame('daily_sales'), 
                               left_index=True, right_index=True, how='left')

In [118]:
daily_sales_metrics = daily_sales_metrics.merge(daily_box_vol.to_frame('vol_usd'), left_index=True, right_index=True, how='inner')
daily_sales_metrics = daily_sales_metrics.merge(cumulative_box_vol.to_frame('cumulative_vol'), left_index=True, right_index=True, how='inner')

In [119]:
daily_sales_metrics

,cumulative_sales,daily_sales,vol_usd,cumulative_vol
day,,,,
2024-02-03 00:00:00+00:00,1,1,33.09,33.09
2024-02-04 00:00:00+00:00,1,0,0.00,33.09
2024-02-05 00:00:00+00:00,1,0,0.00,33.09
2024-02-06 00:00:00+00:00,1,0,0.00,33.09
2024-02-07 00:00:00+00:00,1,0,0.00,33.09
...,...,...,...,...
2024-07-20 00:00:00+00:00,14,0,0.00,"33,482.18"
2024-07-21 00:00:00+00:00,14,0,0.00,"33,482.18"
2024-07-22 00:00:00+00:00,14,0,0.00,"33,482.18"


In [120]:
daily_vol_fig = make_subplots(specs=[[{"secondary_y": True}]])

daily_vol_fig.add_trace(
    go.Bar(
        x=daily_sales_metrics.index,
        y=daily_sales_metrics['vol_usd'],
        name='Sales Volume'
    ),
    secondary_y=False
)

daily_vol_fig.add_trace(
    go.Scatter(
        x=daily_sales_metrics.index,
        y=daily_sales_metrics['cumulative_vol'],
        name='Cumulative Sales Volume',
        mode='lines'
    ),
    secondary_y=True
)

daily_vol_fig.update_xaxes(title_text="Date")

# daily_vol_fig.show()


In [121]:
daily_sales_fig = make_subplots(specs=[[{"secondary_y": True}]])

daily_sales_fig.add_trace(
    go.Bar(
        x=daily_sales_metrics.index,
        y=daily_sales_metrics['daily_sales'],
        name='Sales'
    ),
    secondary_y=False
)

daily_sales_fig.add_trace(
    go.Scatter(
        x=daily_sales_metrics.index,
        y=daily_sales_metrics['cumulative_sales'],
        name='Cumulative Sales',
        mode='lines'
    ),
    secondary_y=True
)

daily_sales_fig.update_xaxes(title_text="Date")

# daily_sales_fig.show()


In [122]:
monthly_listings = monthly_listings.to_frame('listings')


In [123]:
monthly_sales = box_domains_sales['name'].resample('M').count()
monthly_sales = monthly_sales.reset_index()
monthly_sales['day'] = pd.to_datetime(monthly_sales['day']).dt.strftime('%Y-%m-%d')
monthly_sales.set_index('day', inplace=True)

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_9536\2388482189.py:1: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [124]:
monthly_sales

,name
day,
2024-02-29,8
2024-03-31,0
2024-04-30,5
2024-05-31,1
2024-06-30,0
2024-07-31,2


In [125]:
monthly_listings.index

DatetimeIndex(['2024-01-31', '2024-02-29', '2024-03-31', '2024-04-30',
               '2024-05-31', '2024-06-30', '2024-07-31'],
              dtype='datetime64[ns]', name='start_time', freq='ME')

In [126]:
monthly_sales.index = pd.to_datetime(monthly_sales.index)

In [127]:
monthly_sales_reindexed = monthly_sales.reindex(monthly_listings.index).fillna(0)
monthly_sales_reindexed


,name
start_time,
2024-01-31,0.00
2024-02-29,8.00
2024-03-31,0.00
2024-04-30,5.00
2024-05-31,1.00
2024-06-30,0.00
2024-07-31,2.00


In [128]:
monthly_listings['sales'] = monthly_sales_reindexed['name']

In [129]:
monthly_listings

,listings,sales
start_time,,
2024-01-31,25,0.00
2024-02-29,17,8.00
2024-03-31,11,0.00
2024-04-30,88,5.00
2024-05-31,141,1.00
2024-06-30,76,0.00
2024-07-31,143,2.00


In [130]:
monthly_listings['listings_to_sales_ratio'] = monthly_listings['listings'] / monthly_listings['sales']
monthly_listings['listings_to_sales_ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)

monthly_listings

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_9536\3641184892.py:2: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





,listings,sales,listings_to_sales_ratio
start_time,,,
2024-01-31,25,0.00,0.00
2024-02-29,17,8.00,2.12
2024-03-31,11,0.00,0.00
2024-04-30,88,5.00,17.60
2024-05-31,141,1.00,141.00
2024-06-30,76,0.00,0.00
2024-07-31,143,2.00,71.50


In [131]:
listing_to_sales_fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add bar chart for sales
listing_to_sales_fig.add_trace(
    go.Bar(
        x=monthly_listings.index,
        y=monthly_listings['sales'],
        name='Sales'
    ),
    secondary_y=False
)

listing_to_sales_fig.add_trace(
    go.Bar(
        x=monthly_listings.index,
        y=monthly_listings['listings'],
        name='Listings'
    ),
    secondary_y=False
)

# Add line chart for cumulative sales
listing_to_sales_fig.add_trace(
    go.Scatter(
        x=monthly_listings.index,
        y=monthly_listings['listings_to_sales_ratio'],
        name='Listings to Sales Ratio',
        mode='lines'
    ),
    secondary_y=True
)

# listing_to_sales_fig.show()


In [132]:
monthly_mints = box_domains_mints.resample('M').count()
monthly_mints.reset_index(inplace=True)
monthly_mints['day'] = pd.to_datetime(monthly_mints['day']).dt.strftime('%Y-%m-%d') 
print(monthly_mints)

          day  name
0  2023-10-31     2
1  2023-11-30     1
2  2023-12-31    17
3  2024-01-31  1156
4  2024-02-29   245
5  2024-03-31   116
6  2024-04-30   421
7  2024-05-31   281
8  2024-06-30   153
9  2024-07-31  1535


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_9536\2186263740.py:1: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [133]:
monthly_mints.set_index('day', inplace=True)


In [134]:
monthly_mints.index = pd.to_datetime(monthly_mints.index) 
monthly_mints.index

DatetimeIndex(['2023-10-31', '2023-11-30', '2023-12-31', '2024-01-31',
               '2024-02-29', '2024-03-31', '2024-04-30', '2024-05-31',
               '2024-06-30', '2024-07-31'],
              dtype='datetime64[ns]', name='day', freq=None)

In [135]:
monthly_sales_reindexed.index

DatetimeIndex(['2024-01-31', '2024-02-29', '2024-03-31', '2024-04-30',
               '2024-05-31', '2024-06-30', '2024-07-31'],
              dtype='datetime64[ns]', name='start_time', freq='ME')

In [136]:
monthly_mints['sales'] = monthly_sales_reindexed['name'] 
monthly_mints.fillna(0, inplace=True)
monthly_mints['mint_to_sales_ratio'] = monthly_mints['name'] / monthly_mints['sales'] 
monthly_mints['mint_to_sales_ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)
monthly_mints.rename(columns={'name':'mints'}, inplace=True)


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_9536\760772224.py:4: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [137]:
monthly_mints

,mints,sales,mint_to_sales_ratio
day,,,
2023-10-31,2,0.00,0.00
2023-11-30,1,0.00,0.00
2023-12-31,17,0.00,0.00
2024-01-31,1156,0.00,0.00
2024-02-29,245,8.00,30.62
2024-03-31,116,0.00,0.00
2024-04-30,421,5.00,84.20
2024-05-31,281,1.00,281.00
2024-06-30,153,0.00,0.00


In [138]:
mint_to_sales_fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add bar chart for sales
mint_to_sales_fig.add_trace(
    go.Bar(
        x=monthly_mints.index,
        y=monthly_mints['sales'],
        name='Sales'
    ),
    secondary_y=False
)

mint_to_sales_fig.add_trace(
    go.Bar(
        x=monthly_mints.index,
        y=monthly_mints['mints'],
        name='Mints'
    ),
    secondary_y=False
)

# Add line chart for cumulative sales
mint_to_sales_fig.add_trace(
    go.Scatter(
        x=monthly_mints.index,
        y=monthly_mints['mint_to_sales_ratio'],
        name='Mints to Sales Ratio',
        mode='lines'
    ),
    secondary_y=True
)

# mint_to_sales_fig.show()


In [139]:
cumulative_listings_to_sales = total_box_listings / total_box_sales
print(cumulative_listings_to_sales)

31.3125


In [140]:
cumulative_mint_to_sales = total_box_mints / total_box_sales
print(cumulative_mint_to_sales)

245.4375


In [141]:
import os
print(os.getcwd())


e:\Projects\box_app


***Box Domains Valuation Model***

**Data Processing**

domain_path = 'E:/Projects/box_app/data/domain-name-sales.tsv'  
domain_data = pd.read_csv(domain_path, delimiter='\t')

In [142]:
domain_path = 'data/domain-name-sales.tsv'  
domain_data = pd.read_csv(domain_path, delimiter='\t')


In [143]:
domain_data.set_index('date', inplace=True)
domain_data = domain_data.drop(columns=['venue'])
domain_data.sort_index(inplace=True)

In [144]:
domain_data.index = pd.to_datetime(domain_data.index)
domain_data

,domain,price
date,,
1999-04-01,altavista.com,3250000
1999-04-01,bingo.com,1100000
1999-11-01,fly.com,1500000
1999-12-01,autos.com,2200000
1999-12-01,england.com,2000000
...,...,...
2021-01-01,yes.movie,253
2021-01-01,yopal.com,405
2021-01-01,yougraph.com,1161


In [145]:
domain_data['domain_length'] = domain_data['domain'].apply(len)
domain_data['num_vowels'] = domain_data['domain'].apply(lambda x: sum([1 for char in x if char in 'aeiou']))
domain_data['num_consonants'] = domain_data['domain'].apply(lambda x: sum([1 for char in x if char.isalpha() and char not in 'aeiou']))
domain_data['tld'] = domain_data['domain'].apply(lambda x: x.split('.')[-1])  # Extract TLD


In [146]:
domain_data

,domain,price,domain_length,num_vowels,num_consonants,tld
date,,,,,,
1999-04-01,altavista.com,3250000,13,5,7,com
1999-04-01,bingo.com,1100000,9,3,5,com
1999-11-01,fly.com,1500000,7,1,5,com
1999-12-01,autos.com,2200000,9,4,4,com
1999-12-01,england.com,2000000,11,3,7,com
...,...,...,...,...,...,...
2021-01-01,yes.movie,253,9,4,4,movie
2021-01-01,yopal.com,405,9,3,5,com
2021-01-01,yougraph.com,1161,12,4,7,com


In [147]:
box_domains_sales.columns

Index(['name', 'price_usd', 'price_eth'], dtype='object')

In [148]:
filtered_box = box_domains_sales.drop(columns=['price_eth'])
filtered_box.rename(columns={'name':'domain', 'price_usd':'price'}, inplace=True)


In [149]:
filtered_box['domain_length'] = filtered_box['domain'].apply(len)
filtered_box['num_vowels'] = filtered_box['domain'].apply(lambda x: sum([1 for char in x if char in 'aeiou']))
filtered_box['num_consonants'] = filtered_box['domain'].apply(lambda x: sum([1 for char in x if char.isalpha() and char not in 'aeiou']))
filtered_box['tld'] = filtered_box['domain'].apply(lambda x: x.split('.')[-1])  # Extract TLD


In [150]:
filtered_box.index = filtered_box.index.strftime('%Y-%m-%d')

In [151]:
filtered_box

,domain,price,domain_length,num_vowels,num_consonants,tld
day,,,,,,
2024-02-03,696.box,33.09,7,1,2,box
2024-02-10,to.box,96.97,6,2,3,box
2024-02-20,uae.box,203.19,7,4,2,box
2024-02-25,010.box,119.08,7,1,2,box
2024-02-25,ap.box,134.59,6,2,3,box
2024-02-25,40.box,209.36,6,1,2,box
2024-02-25,015.box,104.68,7,1,2,box
2024-02-25,75.box,269.17,6,1,2,box
2024-04-17,mewtwo.box,29.61,10,3,6,box


In [152]:
features = ['domain_length', 'num_vowels', 'num_consonants', 'tld']
X = domain_data[features]
y = domain_data['price']

In [153]:
# Preprocess categorical data (TLD) and handle missing values
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), ['domain_length', 'num_vowels', 'num_consonants']),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), ['tld'])
    ]
)

# Create a pipeline with Ridge regression
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge())
])

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'regressor__alpha': [0.1, 1.0, 10.0, 100.0, 1000.0]
}

In [154]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


**Ridge Regression**

In [155]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

# Best model from grid search
best_model = grid_search.best_estimator_

# Predict and evaluate
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Best Alpha: {grid_search.best_params_["regressor__alpha"]}')
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')

Best Alpha: 1000.0
MAE: 2705.0255351946857
MSE: 2852191402.3463864
R²: 0.0003925999206353392


**Random Forest Regressor**

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=200, max_depth=20, min_samples_split=5, random_state=42))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')

**XGBoost**

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(n_estimators=200, max_depth=5, learning_rate=0.1, random_state=42))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')

**LightGBM**

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LGBMRegressor(n_estimators=200, max_depth=5, learning_rate=0.1, random_state=42))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')

**Cat Boost**

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', CatBoostRegressor(iterations=200, depth=5, learning_rate=0.1, random_state=42, verbose=0))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')

**Prophet**

from sklearn.base import BaseEstimator, TransformerMixin

class ProphetRegressor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.model = Prophet()
        self.fitted_model = None

    def fit(self, X, y=None):
        df = pd.DataFrame({'ds': X.squeeze(), 'y': y})
        self.fitted_model = self.model.fit(df)
        return self

    def predict(self, X):
        future = pd.DataFrame({'ds': X.squeeze()})
        forecast = self.fitted_model.predict(future)
        return forecast['yhat'].values

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', ProphetRegressor())
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



**Best Model**

In [156]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge(alpha=1000.0))  # Set the best alpha value from grid search
])

In [157]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')

MAE: 2705.0255351946857
MSE: 2852191402.3463864
R²: 0.0003925999206353392


In [158]:
filtered_box

,domain,price,domain_length,num_vowels,num_consonants,tld
day,,,,,,
2024-02-03,696.box,33.09,7,1,2,box
2024-02-10,to.box,96.97,6,2,3,box
2024-02-20,uae.box,203.19,7,4,2,box
2024-02-25,010.box,119.08,7,1,2,box
2024-02-25,ap.box,134.59,6,2,3,box
2024-02-25,40.box,209.36,6,1,2,box
2024-02-25,015.box,104.68,7,1,2,box
2024-02-25,75.box,269.17,6,1,2,box
2024-04-17,mewtwo.box,29.61,10,3,6,box


In [159]:
box_X = filtered_box[features]

# Predict prices for .box domains using the best model
filtered_box['predicted_price'] = pipeline.predict(box_X)

print(filtered_box[['domain', 'predicted_price']])

                         domain  predicted_price
day                                             
2024-02-03              696.box         1,168.93
2024-02-10               to.box         3,066.39
2024-02-20              uae.box         3,180.12
2024-02-25              010.box         1,168.93
2024-02-25               ap.box         3,066.39
2024-02-25               40.box         2,005.65
2024-02-25              015.box         1,168.93
2024-02-25               75.box         2,005.65
2024-04-17           mewtwo.box         1,560.93
2024-04-24             0000.box           332.21
2024-04-27               vm.box         2,786.34
2024-04-28              man.box         2,620.01
2024-04-28             good.box         2,453.69
2024-05-07             onyx.box         2,173.63
2024-07-23       lauc.chain.box           -54.82
2024-07-24  cryptobet.chain.box        -2,286.72


In [160]:
r2 = r2_score(filtered_box['price'], filtered_box['predicted_price'])
print(f'r2 {r2}')

r2 0.043351818604011605


**.Box Domain Valuator**

In [161]:
filtered_box_2 = filtered_box.drop(columns=['predicted_price'])
filtered_box_2

,domain,price,domain_length,num_vowels,num_consonants,tld
day,,,,,,
2024-02-03,696.box,33.09,7,1,2,box
2024-02-10,to.box,96.97,6,2,3,box
2024-02-20,uae.box,203.19,7,4,2,box
2024-02-25,010.box,119.08,7,1,2,box
2024-02-25,ap.box,134.59,6,2,3,box
2024-02-25,40.box,209.36,6,1,2,box
2024-02-25,015.box,104.68,7,1,2,box
2024-02-25,75.box,269.17,6,1,2,box
2024-04-17,mewtwo.box,29.61,10,3,6,box


In [162]:
combined_data = pd.concat([domain_data, filtered_box_2], ignore_index=True)

In [163]:
combined_data

,domain,price,domain_length,num_vowels,num_consonants,tld
0,altavista.com,"3,250,000.00",13,5,7,com
1,bingo.com,"1,100,000.00",9,3,5,com
2,fly.com,"1,500,000.00",7,1,5,com
3,autos.com,"2,200,000.00",9,4,4,com
4,england.com,"2,000,000.00",11,3,7,com
...,...,...,...,...,...,...
348247,man.box,224.60,7,2,4,box
348248,good.box,314.61,8,3,4,box
348249,onyx.box,167.83,8,2,5,box
348250,lauc.chain.box,5.00,14,5,7,box


In [164]:
X = combined_data[features]
y = combined_data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')


MAE: 2722.339752286588
MSE: 2865321236.369076
R²: 0.00044123600329526447


In [165]:
def model_prep(data):
    data['domain_length'] = data['domain'].apply(len)
    data['num_vowels'] = data['domain'].apply(lambda x: sum([1 for char in x if char in 'aeiou']))
    data['num_consonants'] = data['domain'].apply(lambda x: sum([1 for char in x if char.isalpha() and char not in 'aeiou']))
    data['tld'] = data['domain'].apply(lambda x: x.split('.')[-1]) 
    return data

In [166]:
def value_domain(domain):
    domain_x = domain[features]
    value = pipeline.predict(domain_x)
    print(f'predicted value: {value[0]}')
    return value[0] 

In [167]:
test_domain = 'eth.box' ## for model, just have the person input before .box, have it automatically add .box
test_domain_df = pd.DataFrame({'domain': [test_domain]})
test_domain_processed = model_prep(test_domain_df)
test_domain_value = value_domain(test_domain_processed)

predicted value: 2580.153071802707


In [168]:
test_domain_value

np.float64(2580.153071802707)

***Dash App***

Domain valuator would be callback

In [169]:
latest_box_domains_sales.reset_index(inplace=True)

In [170]:
latest_box_domains_sales.sort_values(by='day', ascending=False, inplace=True)

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_9536\3336277017.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [171]:
latest_box_domains_mints.reset_index(inplace=True)

In [172]:
latest_box_domains_mints.sort_values(by='day', ascending=False, inplace=True)

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_9536\4287417152.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [173]:
avg_box_sale = box_domains_sales['price_usd'].mean()

In [174]:
highest_selling_domains = box_domains_sales[['name','price_usd','price_eth']].sort_values(by='price_usd', ascending=False)
highest_selling_domains = highest_selling_domains.head(10)
highest_selling_domains

,name,price_usd,price_eth
day,,,
2024-04-27 16:00:00+00:00,vm.box,"31,481.47",10.00
2024-04-28 07:00:00+00:00,good.box,314.61,0.10
2024-02-25 03:00:00+00:00,75.box,269.17,0.09
2024-04-28 01:00:00+00:00,man.box,224.60,0.07
2024-02-25 03:00:00+00:00,40.box,209.36,0.07
2024-02-20 12:00:00+00:00,uae.box,203.19,0.07
2024-05-07 22:00:00+00:00,onyx.box,167.83,0.06
2024-02-25 03:00:00+00:00,ap.box,134.59,0.04
2024-02-25 02:00:00+00:00,010.box,119.08,0.04


In [175]:
monthly_box_sales_metrics['cumulative_volume'] = monthly_box_sales_metrics['volume_usd'].cumsum()
monthly_box_sales_metrics.reset_index(inplace=True) 
monthly_box_sales_metrics.sort_values(by='day', ascending=False, inplace=True)

In [176]:
monthly_box_sales_metrics['cumulative domains sold'] = monthly_box_sales_metrics['domains sold'].cumsum()

In [177]:
historical_listing_to_sales = closest_listings[['name','bid_event_timestamp','sale_event_timestamp',
                                                'list_usd','sale_usd','percent_change']]
historical_listing_to_sales.sort_values(by='bid_event_timestamp', ascending=False, inplace=True)
historical_listing_to_sales

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_9536\2820302304.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,name,bid_event_timestamp,sale_event_timestamp,list_usd,sale_usd,percent_change


In [178]:
box_listing_data.columns

Index(['listings', 'max_price', 'min_price', 'avg_price'], dtype='object')

In [179]:
box_listing_data.tail(20)

,listings,max_price,min_price,avg_price
start_time,,,,
2024-07-05,1,"8,581.74","8,581.74","8,581.74"
2024-07-06,7,"30,690.04","23,780.08","25,754.35"
2024-07-07,1,"39,241.41","39,241.41","39,241.41"
2024-07-08,3,"8,984.76",305.61,"6,091.71"
2024-07-09,7,"101,282.94","2,127.33","26,487.11"
2024-07-10,2,"155,186.00","155,186.00","155,186.00"
2024-07-11,1,"28,950.90","28,950.90","28,950.90"
2024-07-12,3,"31,185.30","3,110.88","14,030.84"
2024-07-13,1,"9,477.42","9,477.42","9,477.42"


In [180]:
highest_selling_domains

,name,price_usd,price_eth
day,,,
2024-04-27 16:00:00+00:00,vm.box,"31,481.47",10.00
2024-04-28 07:00:00+00:00,good.box,314.61,0.10
2024-02-25 03:00:00+00:00,75.box,269.17,0.09
2024-04-28 01:00:00+00:00,man.box,224.60,0.07
2024-02-25 03:00:00+00:00,40.box,209.36,0.07
2024-02-20 12:00:00+00:00,uae.box,203.19,0.07
2024-05-07 22:00:00+00:00,onyx.box,167.83,0.06
2024-02-25 03:00:00+00:00,ap.box,134.59,0.04
2024-02-25 02:00:00+00:00,010.box,119.08,0.04


In [181]:
highest_selling_domains_fig = px.bar(highest_selling_domains, x=highest_selling_domains['name'], y=highest_selling_domains['price_usd'],
                                     title='10 Highest Selling Domains')
# highest_selling_domains_fig.show()

In [182]:


box_listing_data_fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add bar chart for sales
# box_listing_data_fig.add_trace(
#     go.Bar(
#         x=box_listing_data.index,
#         y=box_listing_data['max_price'],
#         name='Max Price'
#     ),
#     secondary_y=False
# )

# box_listing_data_fig.add_trace(
#     go.Bar(
#         x=box_listing_data.index,
#         y=box_listing_data['min_price'],
#         name='Min Price'
#     ),
#     secondary_y=False
# )

# Add line chart for cumulative sales
box_listing_data_fig.add_trace(
    go.Bar(
        x=box_listing_data.index,
        y=box_listing_data['avg_price'],
        name='Avg Price',
    ),
    secondary_y=False
)

box_listing_data_fig.add_trace(
    go.Scatter(
        x=box_listing_data.index,
        y=box_listing_data['listings'],
        name='Listings',
        mode='lines'
    ),
    secondary_y=True
)

# box_listing_data_fig.show()


In [183]:
box_listings.drop(columns=['tokenid','eth_usd'], inplace=True)


In [184]:
latest_box_listings = box_listings.sort_index(ascending=False)
latest_box_listings = latest_box_listings.head(10)
latest_box_listings.reset_index(inplace=True)

In [185]:
mint_to_sales_fig.update_layout(
    plot_bgcolor='#fafafa',
    paper_bgcolor='#fafafa',
    title="Mints to Sales Metrics"
)

listing_to_sales_fig.update_layout(
    plot_bgcolor='#fafafa',
    paper_bgcolor='#fafafa',
    title="Listings to Sales Metrics"
)

daily_box_mints_fig.update_layout(
    plot_bgcolor='#fafafa',
    paper_bgcolor='#fafafa',
    title="Daily Mints"
)

daily_box_sales_fig.update_layout(
    plot_bgcolor='#fafafa',
    paper_bgcolor='#fafafa',
    title="Daily Sales"
)

daily_vol_fig.update_layout(
    plot_bgcolor='#fafafa',
    paper_bgcolor='#fafafa',
    title="Daily Volume"
)

daily_sales_fig.update_layout(
    plot_bgcolor='#fafafa',
    paper_bgcolor='#fafafa',
    title="Daily Sales Metrics"
)

daily_mint_metrics_fig.update_layout(
    plot_bgcolor='#fafafa',
    paper_bgcolor='#fafafa',
    title="Daily Mints Metrics"
)

listings_growth_rate_fig.update_layout(
    plot_bgcolor='#fafafa',
    paper_bgcolor='#fafafa',
    title="Listings Growth Rate"
)

highest_selling_domains_fig.update_layout(
    plot_bgcolor='#fafafa',
    paper_bgcolor='#fafafa',
)



Average Time to Sell a .box Domain: 19 days 11:19:25.506097561

Average Listing Price to Sale Price Change -14.673687659116787

Cumulative Listings to Sales Ratio: 25:1

Cumulative Mints to Sales Ratio: 171:1

Monthly Listings Growth Rate: -70.2247191011236

In [186]:
avg_time_to_sell

NaT

In [187]:
listing_price_to_sale_avg_pct_change = 0 if pd.isna(listing_price_to_sale_avg_pct_change) else listing_price_to_sale_avg_pct_change
# avg_time_to_sell = pd.to_Date0 if pd.isna(avg_time_to_sell) else avg_time_to_sell

In [188]:
key_metrics = [
    {"label": "Average Listing Price to Sale Price Change", "value": f"{round(int(listing_price_to_sale_avg_pct_change),0)}", "unit": "%"},
    {"label": "Average Days on Market", "value": str(avg_time_to_sell.days), "unit": " days"},
    {"label": "Cumulative Listings to Sales Ratio", "value": f"{round(int(cumulative_listings_to_sales), 0)}", "unit": ":1"},
    {"label": "Cumulative Mints to Sales Ratio", "value": f"{round(int(cumulative_mint_to_sales),0)}", "unit": ":1"},
    {"label": "Monthly Listings Growth Rate", "value": f"{listings_growth_rate.iloc[-1]:.2f}", "unit": "%"}
]


In [189]:
max_eth_sale_row

name           vm.box
price_usd   31,481.47
price_eth       10.00
Name: 2024-04-27 16:00:00+00:00, dtype: object

In [190]:
max_eth_sale_details = {
    "name": max_eth_sale_row["name"],
    "price_usd": max_eth_sale_row["price_usd"],
    "price_eth": max_eth_sale_row["price_eth"],
    "date": max_eth_sale_row.name  # This is the index (timestamp)
}

highest_sold_domain_str = f"""
Name: {max_eth_sale_details['name']}
Price (USD): ${max_eth_sale_details['price_usd']:.2f}
Price (ETH): {max_eth_sale_details['price_eth']}
Date: {max_eth_sale_details['date'].strftime('%Y-%m-%d')}
"""

In [191]:
highest_sold_domain_str

'\nName: vm.box\nPrice (USD): $31481.47\nPrice (ETH): 10.0\nDate: 2024-04-27\n'

In [192]:
sales_metrics = [
    {"label": "Total Sales Volume ", "value":f"${cumulative_box_vol.iloc[-1]:,.2f}", "unit": ""},
    {"label": "Highest Sold Domain", "value": highest_sold_domain_str, "unit": ""},
    {"label": "Average Sales Price ", "value":f"${avg_box_sale:,.2f}", "unit": ""}
    
]

In [193]:
listings_metrics = [

    {"label":"Total .box Listings on Opensea ", "value": str(total_box_listings), "unit": ""}
]

In [194]:
mints_metrics = [
    {"label":"Total .box Mints ", "value":str(total_box_mints), "unit": ""}
]

In [195]:
def generate_table(dataframe, max_rows=11):
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in dataframe.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))
        ])
    ])

In [196]:
latest_box_listings

,start_time,price_in_eth,end_time,name,price_in_usd_start_time
0,2024-07-24 17:00:00,0.00,2025-01-20 17:00:00,girls-on.chain.box,2.71
1,2024-07-24 17:00:00,0.00,2025-01-20 17:00:00,memecoins-on.chain.box,2.71
2,2024-07-24 17:00:00,0.00,2025-01-20 17:00:00,memes.super.box,2.71
3,2024-07-24 17:00:00,0.00,2025-01-19 06:00:00,memes.super.box,2.74
4,2024-07-24 17:00:00,0.00,2025-01-19 03:00:00,memecoins-on.chain.box,2.74
5,2024-07-24 17:00:00,0.00,2025-01-19 03:00:00,girls-on.chain.box,2.74
6,2024-07-24 12:00:00,1.50,2024-08-24 12:00:00,price.box,"5,199.94"
7,2024-07-24 06:00:00,0.00,2024-08-24 06:00:00,cryptobet.chain.box,3.09
8,2024-07-24 00:00:00,0.01,2024-08-24 00:00:00,bluestate.chain.box,49.98
9,2024-07-23 23:00:00,0.00,2024-08-23 23:00:00,62c9ec.chain.box,5.00


external_stylesheets = [
    'https://cdnjs.cloudflare.com/ajax/libs/normalize/8.0.1/normalize.min.css', 
    '/assets/styles.css'
]

app = Dash(__name__, external_stylesheets=external_stylesheets)

tts_markdown = f"""
Average Time to Sell a .box Domain: {avg_time_to_sell}
"""

listing_to_sale_mk = f"""Average Listing Price to Sale Price Change {listing_price_to_sale_avg_pct_change} """

cumulative_lts_mk = f"""Cumulative Listings to Sales Ratio: {round(int(cumulative_listings_to_sales), 0)}:1"""
cumulative_mts_mk = f"""Cumulative Mints to Sales Ratio: {round(int(cumulative_mint_to_sales),0)}:1"""
cum_sale_latest_mk = f"""Total Sales Volume: {cumulative_box_vol.iloc[-1]} """
max_sale_mk = f"""Highest Sold Domain: {max_eth_sale_row} """
total_mints_mk = f"""Total .box Mints: {total_box_mints} """
total_listings_mk = f"""Total .box Listings on Opensea {total_box_listings} """
latest_listings_growth_mk = f"""Monthly Listings Growth Rate: {listings_growth_rate.iloc[-1]} """
avg_sold_mk = f"""Average Sales Price: {avg_box_sale} """





# Define the layout of the app
app.layout = html.Div(style={'backgroundColor': 'var(--color-background)'}, children=[
    html.H1(children='.box Domains Dashboard', style={'textAlign': 'center', 'color': 'var(--wcm-color-fg-1)'}),
    html.Br(),
    html.H2('.box Domain Valuator', style={'color': 'var(--wcm-color-fg-2)'}),
    html.Div([
        "Input Domain Name: ",
        dcc.Input(id='valuator-input', value='example.box', type='text'),
        html.Button('Submit', id='submit-button', n_clicks=0)
    ], style={'color': 'var(--wcm-color-fg-1)'}),
    html.Br(),
    html.Div(id='valuator-output', style={'color': 'var(--wcm-color-fg-1)'}),
    html.Br(),
    html.H2('Key Metrics'),
    html.Div(className='metrics-container', style={
        'display': 'grid',
        'gridTemplateColumns': 'repeat(3, 1fr)',  # Adjust based on the number of metrics
        'gap': '20px',
        'padding': '20px',
        'backgroundColor': 'var(--wcm-color-bg-1)',
        'borderRadius': 'var(--wcm-container-border-radius)',
        'boxShadow': '0 4px 8px var(--wcm-color-overlay)'
    }, children=[
        html.Div(className='metric', style={
            'padding': '15px',
            'backgroundColor': 'var(--wcm-color-bg-2)',
            'borderRadius': 'var(--wcm-button-border-radius)',
            'textAlign': 'center',
            'color': 'var(--wcm-color-fg-1)'
        }, children=[
            html.Span(metric["label"], style={'color': 'var(--wcm-color-fg-2)'}),
            html.Span(f"{metric['value']}{metric['unit']}", style={'color': 'var(--wcm-accent-color)'})
        ]) for metric in key_metrics
    ]),
    dcc.Graph(id='mint to sales', figure=mint_to_sales_fig),
    dcc.Graph(id='listings to sales', figure=listing_to_sales_fig),
    
    html.H2('Sales'),
    html.Div(className='metrics-container', style={
        'display': 'grid',
        'gridTemplateColumns': 'repeat(3, 1fr)',  # Adjust based on the number of metrics
        'gap': '20px',
        'padding': '20px',
        'backgroundColor': 'var(--wcm-color-bg-1)',
        'borderRadius': 'var(--wcm-container-border-radius)',
        'boxShadow': '0 4px 8px var(--wcm-color-overlay)'
    }, children=[
        html.Div(className='metric', style={
            'padding': '15px',
            'backgroundColor': 'var(--wcm-color-bg-2)',
            'borderRadius': 'var(--wcm-button-border-radius)',
            'textAlign': 'center',
            'color': 'var(--wcm-color-fg-1)'
        }, children=[
            html.Span(metric["label"], style={'color': 'var(--wcm-color-fg-2)'}),
            html.Span(f"{metric['value']}{metric['unit']}", style={'color': 'var(--wcm-accent-color)'})
        ]) for metric in sales_metrics
    ]),
    dcc.Graph(id='daily_sales_count', figure=daily_sales_fig),
    dcc.Graph(id='daily_sales_vol', figure=daily_vol_fig),
    dcc.Graph(id='highest selling', figure=highest_selling_domains_fig),
    html.Br(),
    html.H3('Monthly Sales Metrics'),
    # generate_table(monthly_box_sales_metrics),
    dash_table.DataTable(
        id='monthly_sales_metrics',
        columns=[{"name": i, "id": i} for i in monthly_box_sales_metrics.columns],
        data=monthly_box_sales_metrics.to_dict('records'),
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
            'whiteSpace': 'normal',
            'font-family': 'var(--font-primary, "Inter")',
            'backgroundColor': 'var(--color-background)',
            'color': 'var(--wcm-color-fg-2)'
        }
    ),

    html.H3('10 Latest Sales'),
    dash_table.DataTable(
        id='latest_sales',
        columns=[{"name": i, "id": i} for i in latest_box_domains_sales.columns],
        data=latest_box_domains_sales.to_dict('records'),
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
            'whiteSpace': 'normal',
            'font-family': 'var(--font-primary, "Inter")',
            'backgroundColor': 'var(--color-background)',
            'color': 'var(--wcm-color-fg-2)'
        }
    ),
    html.Br(),
    html.H3('10 Highest Selling Domains'),
    dash_table.DataTable(
        id='highest_sales',
        columns=[{"name": i, "id": i} for i in highest_selling_domains.columns],
        data=highest_selling_domains.to_dict('records'),
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
            'whiteSpace': 'normal',
            'font-family': 'var(--font-primary, "Inter")',
            'backgroundColor': 'var(--color-background)',
            'color': 'var(--wcm-color-fg-2)'
        }
    ),
    
    html.Br(),
    html.H2('Listings'),
    html.Div(className='metrics-container', style={
        'display': 'grid',
        'gridTemplateColumns': 'repeat(1, 1fr)',  # Adjust based on the number of metrics
        'gap': '20px',
        'padding': '20px',
        'backgroundColor': 'var(--wcm-color-bg-1)',
        'borderRadius': 'var(--wcm-container-border-radius)',
        'boxShadow': '0 4px 8px var(--wcm-color-overlay)'
    }, children=[
        html.Div(className='metric', style={
            'padding': '15px',
            'backgroundColor': 'var(--wcm-color-bg-2)',
            'borderRadius': 'var(--wcm-button-border-radius)',
            'textAlign': 'center',
            'color': 'var(--wcm-color-fg-1)'
        }, children=[
            html.Span(metric["label"], style={'color': 'var(--wcm-color-fg-2)'}),
            html.Span(f"{metric['value']}{metric['unit']}", style={'color': 'var(--wcm-accent-color)'})
        ]) for metric in listings_metrics
    ]),
    dcc.Graph(id='monthly listings growth', figure=listings_growth_rate_fig),
    html.H3('Historical Listings to Sales'),
    dash_table.DataTable(
        id='listings_to_sales',
        columns=[{"name": i, "id": i} for i in historical_listing_to_sales.columns],
        data=historical_listing_to_sales.to_dict('records'),
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
            'whiteSpace': 'normal',
            'font-family': 'var(--font-primary, "Inter")',
            'backgroundColor': 'var(--color-background)',
            'color': 'var(--wcm-color-fg-2)'
        }
    ),
    html.H3('10 Latest Listings'),
    dash_table.DataTable(
        id='latest_listings',
        columns=[{"name": i, "id": i} for i in latest_box_listings.columns],
        data=latest_box_listings.to_dict('records'),
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
            'whiteSpace': 'normal',
            'font-family': 'var(--font-primary, "Inter")',
            'backgroundColor': 'var(--color-background)',
            'color': 'var(--wcm-color-fg-2)'
        }
    ),



    html.H2('Mints'),
    html.Div(className='metrics-container', style={
        'display': 'grid',
        'gridTemplateColumns': 'repeat(1, 1fr)',  # Adjust based on the number of metrics
        'gap': '20px',
        'padding': '20px',
        'backgroundColor': 'var(--wcm-color-bg-1)',
        'borderRadius': 'var(--wcm-container-border-radius)',
        'boxShadow': '0 4px 8px var(--wcm-color-overlay)'
    }, children=[
        html.Div(className='metric', style={
            'padding': '15px',
            'backgroundColor': 'var(--wcm-color-bg-2)',
            'borderRadius': 'var(--wcm-button-border-radius)',
            'textAlign': 'center',
            'color': 'var(--wcm-color-fg-1)'
        }, children=[
            html.Span(metric["label"], style={'color': 'var(--wcm-color-fg-2)'}),
            html.Span(f"{metric['value']}{metric['unit']}", style={'color': 'var(--wcm-accent-color)'})
        ]) for metric in mints_metrics
    ]),
    dcc.Graph(id='daily_mints', figure=daily_mint_metrics_fig),
    html.Br(),
    html.H3('10 Latest Mints'),
    dash_table.DataTable(
        id='latest_mints',
        columns=[{"name": i, "id": i} for i in latest_box_domains_mints.columns],
        data=latest_box_domains_mints.to_dict('records'),
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
            'whiteSpace': 'normal',
            'font-family': 'var(--font-primary, "Inter")',
            'backgroundColor': 'var(--color-background)',
            'color': 'var(--wcm-color-fg-2)'
        }
    ),
    
    
])

# Define the callback
@callback(
    Output(component_id='valuator-output', component_property='children'),
    Input(component_id='submit-button', component_property='n_clicks'),
    State(component_id='valuator-input', component_property='value')
)
def update_output_div(n_clicks, domain):
    if n_clicks == 0:
        return "Please enter a domain name and click Submit."
    
    if not domain:
        return "Please enter a domain name."
    
    domain_df = pd.DataFrame({'domain': [domain]})
    domain_processed = model_prep(domain_df)
    domain_value = value_domain(domain_processed)
    return f'Estimated Value: ${round(domain_value,2):,.2f}'

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [197]:
external_stylesheets = [
    'https://cdnjs.cloudflare.com/ajax/libs/normalize/8.0.1/normalize.min.css', 
    '/assets/styles.css'
]

app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(style={'backgroundColor': 'var(--color-background)'}, children=[
    html.H1(
        children='.box Domains Dashboard',
        style={
            'textAlign': 'center',
            'color': 'var(--wcm-color-fg-1)',
            'fontSize': '36px',
            'fontWeight': 'bold',
            'marginBottom': '20px'
        }
    ),
    html.Br(),
    html.H2('.box Domain Valuator', style={
        'color': 'var(--wcm-color-fg-2)', 
        'textAlign': 'center', 
        'marginBottom': '20px'
    }),
    html.Div([
        html.Label("Input Domain Name:", style={
            'color': 'var(--wcm-color-fg-1)', 
            'marginRight': '10px',
            'fontWeight': 'bold'
        }),
        dcc.Input(
            id='valuator-input',
            value='example',
            type='text',
            style={
                'padding': '10px',
                'borderRadius': 'var(--wcm-input-border-radius)',
                'border': '1px solid var(--color-border)',
                'marginRight': '10px'
            },
            pattern='[^.]*'  # Regex pattern to disallow '.' character
        ),
        html.Button('Submit', id='submit-button', n_clicks=0, style={
            'padding': '10px 20px', 
            'borderRadius': 'var(--wcm-button-border-radius)', 
            'backgroundColor': 'var(--wcm-accent-color)', 
            'color': 'var(--wcm-accent-fill-color)',
            'border': 'none',
            'cursor': 'pointer'
        })
    ], style={
        'display': 'flex', 
        'alignItems': 'center', 
        'justifyContent': 'center', 
        'marginBottom': '20px'
    }),
    html.Br(),
    html.Div(id='valuator-output', style={
        'color': 'var(--wcm-color-fg-1)', 
        'textAlign': 'center', 
        'marginTop': '20px'
    }),
    html.Br(),
    html.H2('Key Metrics', style={'color': 'var(--wcm-color-fg-1)'}),
    html.Div(className='metrics-container', children=[
        html.Div(className='metric', children=[
            html.Span(metric["label"], className='label'),
            html.Span(f"{metric['value']}{metric['unit']}", className='value')
        ]) for metric in key_metrics
    ]),
    html.Br(),
    dcc.Graph(id='mint to sales', figure=mint_to_sales_fig),
    dcc.Graph(id='listings to sales', figure=listing_to_sales_fig),
    
    html.H2('Sales', style={'color': 'var(--wcm-color-fg-1)'}),
    html.Div(className='metrics-container', children=[
        html.Div(className='metric', children=[
            html.Span(metric["label"], className='label'),
            html.Span(f"{metric['value']}{metric['unit']}", className='value')
        ]) for metric in sales_metrics
    ]),
    html.Br(),
    dcc.Graph(id='daily_sales_count', figure=daily_sales_fig),
    dcc.Graph(id='daily_sales_vol', figure=daily_vol_fig),
    dcc.Graph(id='highest selling', figure=highest_selling_domains_fig),
    html.Br(),
    html.H3('Monthly Sales Metrics', style={'color': 'var(--wcm-color-fg-1)'}),
    html.Div(style={'display': 'flex', 'justify-content': 'center', 'padding': '10px'}, children=[
        html.Div(style={'width': '80%', 'max-width': '1000px'}, children=[
            dash_table.DataTable(
                id='monthly_sales_metrics',
                columns=[{"name": i, "id": i} for i in monthly_box_sales_metrics.columns],
                data=monthly_box_sales_metrics.to_dict('records'),
                style_table={'overflowX': 'auto'},
                style_as_list_view=True,
                style_header={
                    'backgroundColor': 'var(--wcm-color-bg-2)',
                    'fontWeight': 'bold',
                    'color': 'var(--wcm-color-fg-1)'
                },
                style_cell={
                    'height': 'auto',
                    'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
                    'whiteSpace': 'normal',
                    'font-family': 'var(--font-primary, "Inter")',
                    'backgroundColor': 'var(--color-background)',
                    'color': 'var(--wcm-color-fg-2)',
                    'padding': '10px',
                    'border': '1px solid var(--color-border)'
                },
                style_data={
                    'border': '1px solid var(--color-border)',
                    'padding': '10px',
                }
            )
        ])
    ]),
    html.Br(),
    html.H3('10 Latest Sales', style={'color': 'var(--wcm-color-fg-1)'}),
    html.Div(style={'display': 'flex', 'justify-content': 'center', 'padding': '10px'}, children=[
        html.Div(style={'width': '80%', 'max-width': '1000px'}, children=[
            dash_table.DataTable(
                id='latest_sales',
                columns=[{"name": i, "id": i} for i in latest_box_domains_sales.columns],
                data=latest_box_domains_sales.to_dict('records'),
                style_table={'overflowX': 'auto'},
                style_as_list_view=True,
                style_header={
                    'backgroundColor': 'var(--wcm-color-bg-2)',
                    'fontWeight': 'bold',
                    'color': 'var(--wcm-color-fg-1)'
                },
                style_cell={
                    'height': 'auto',
                    'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
                    'whiteSpace': 'normal',
                    'font-family': 'var(--font-primary, "Inter")',
                    'backgroundColor': 'var(--color-background)',
                    'color': 'var(--wcm-color-fg-2)',
                    'padding': '10px',
                    'border': '1px solid var(--color-border)'
                },
                style_data={
                    'border': '1px solid var(--color-border)',
                    'padding': '10px',
                }
            )
        ])
    ]),
    html.Br(),
    html.H3('10 Highest Selling Domains', style={'color': 'var(--wcm-color-fg-1)'}),
    html.Div(style={'display': 'flex', 'justify-content': 'center', 'padding': '10px'}, children=[
        html.Div(style={'width': '80%', 'max-width': '1000px'}, children=[
            dash_table.DataTable(
                id='highest_sales',
                columns=[{"name": i, "id": i} for i in highest_selling_domains.columns],
                data=highest_selling_domains.to_dict('records'),
                style_table={'overflowX': 'auto'},
                style_as_list_view=True,
                style_header={
                    'backgroundColor': 'var(--wcm-color-bg-2)',
                    'fontWeight': 'bold',
                    'color': 'var(--wcm-color-fg-1)'
                },
                style_cell={
                    'height': 'auto',
                    'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
                    'whiteSpace': 'normal',
                    'font-family': 'var(--font-primary, "Inter")',
                    'backgroundColor': 'var(--color-background)',
                    'color': 'var(--wcm-color-fg-2)',
                    'padding': '10px',
                    'border': '1px solid var(--color-border)'
                },
                style_data={
                    'border': '1px solid var(--color-border)',
                    'padding': '10px',
                }
            )
        ])
    ]),
    html.Br(),
    html.H2('Listings', style={'color': 'var(--wcm-color-fg-1)'}),
    html.Div(className='metrics-container', children=[
        html.Div(className='metric', children=[
            html.Span(metric["label"], className='label'),
            html.Span(f"{metric['value']}{metric['unit']}", className='value')
        ]) for metric in listings_metrics
    ]),
    html.Br(),
    dcc.Graph(id='monthly listings growth', figure=listings_growth_rate_fig),
    html.H3('Historical Listings to Sales', style={'color': 'var(--wcm-color-fg-1)'}),
    html.Div(style={'display': 'flex', 'justify-content': 'center', 'padding': '10px'}, children=[
        html.Div(style={'width': '80%', 'max-width': '1000px'}, children=[
            dash_table.DataTable(
                id='listings_to_sales',
                columns=[{"name": i, "id": i} for i in historical_listing_to_sales.columns],
                data=historical_listing_to_sales.to_dict('records'),
                style_table={'overflowX': 'auto'},
                style_as_list_view=True,
                style_header={
                    'backgroundColor': 'var(--wcm-color-bg-2)',
                    'fontWeight': 'bold',
                    'color': 'var(--wcm-color-fg-1)'
                },
                style_cell={
                    'height': 'auto',
                    'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
                    'whiteSpace': 'normal',
                    'font-family': 'var(--font-primary, "Inter")',
                    'backgroundColor': 'var(--color-background)',
                    'color': 'var(--wcm-color-fg-2)',
                    'padding': '10px',
                    'border': '1px solid var(--color-border)'
                },
                style_data={
                    'border': '1px solid var(--color-border)',
                    'padding': '10px',
                }
            )
        ])
    ]),
    html.H3('10 Latest Listings', style={'color': 'var(--wcm-color-fg-1)'}),
    html.Div(style={'display': 'flex', 'justify-content': 'center', 'padding': '10px'}, children=[
        html.Div(style={'width': '80%', 'max-width': '1000px'}, children=[
            dash_table.DataTable(
                id='latest_listings',
                columns=[{"name": i, "id": i} for i in latest_box_listings.columns],
                data=latest_box_listings.to_dict('records'),
                style_table={'overflowX': 'auto'},
                style_as_list_view=True,
                style_header={
                    'backgroundColor': 'var(--wcm-color-bg-2)',
                    'fontWeight': 'bold',
                    'color': 'var(--wcm-color-fg-1)'
                },
                style_cell={
                    'height': 'auto',
                    'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
                    'whiteSpace': 'normal',
                    'font-family': 'var(--font-primary, "Inter")',
                    'backgroundColor': 'var(--color-background)',
                    'color': 'var(--wcm-color-fg-2)',
                    'padding': '10px',
                    'border': '1px solid var(--color-border)'
                },
                style_data={
                    'border': '1px solid var(--color-border)',
                    'padding': '10px',
                }
            )
        ])
    ]),

    html.H2('Mints', style={'color': 'var(--wcm-color-fg-1)'}),
    html.Div(className='metrics-container', children=[
        html.Div(className='metric', children=[
            html.Span(metric["label"], className='label'),
            html.Span(f"{metric['value']}{metric['unit']}", className='value')
        ]) for metric in mints_metrics
    ]),
    html.Br(),
    dcc.Graph(id='daily_mints', figure=daily_mint_metrics_fig),
    html.Br(),
    html.H3('10 Latest Mints', style={'color': 'var(--wcm-color-fg-1)'}),
    html.Div(style={'display': 'flex', 'justify-content': 'center', 'padding': '10px'}, children=[
        html.Div(style={'width': '80%', 'max-width': '1000px'}, children=[
            dash_table.DataTable(
                id='latest_mints',
                columns=[{"name": i, "id": i} for i in latest_box_domains_mints.columns],
                data=latest_box_domains_mints.to_dict('records'),
                style_table={'overflowX': 'auto'},
                style_as_list_view=True,
                style_header={
                    'backgroundColor': 'var(--wcm-color-bg-2)',
                    'fontWeight': 'bold',
                    'color': 'var(--wcm-color-fg-1)'
                },
                style_cell={
                    'height': 'auto',
                    'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
                    'whiteSpace': 'normal',
                    'font-family': 'var(--font-primary, "Inter")',
                    'backgroundColor': 'var(--color-background)',
                    'color': 'var(--wcm-color-fg-2)',
                    'padding': '10px',
                    'border': '1px solid var(--color-border)'
                },
                style_data={
                    'border': '1px solid var(--color-border)',
                    'padding': '10px',
                }
            )
        ])
    ]),
])

# Define the callback
@callback(
    Output(component_id='valuator-output', component_property='children'),
    Input(component_id='submit-button', component_property='n_clicks'),
    State(component_id='valuator-input', component_property='value')
)
def update_output_div(n_clicks, domain_prefix):
    if n_clicks == 0:
        return "Please enter a domain prefix and click Submit."
    
    if not domain_prefix:
        return "Please enter a domain prefix."
    
    if '.' in domain_prefix:
        return "Invalid input. Please enter a valid domain prefix without a '.' character."
    
    domain = f"{domain_prefix}.box"
    domain_df = pd.DataFrame({'domain': [domain]})
    domain_processed = model_prep(domain_df)
    domain_value = value_domain(domain_processed)
    return html.Div([
        html.Div(f'Domain: {domain}', style={'font-weight': 'bold'}),
        html.Div(f'Estimated Value: ${round(domain_value, 2):,.2f}')
    ])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
